In [ ]:
### Packages
rm(list=ls())
graphics.off()
library(xts)
library(zoo)
library(ConnectednessApproach)
library(tseries)
library(FinTS)
library(urca)
library(corrplot)
library(vars)
library(MTS)
library(readxl)
library(qgraph)
library(tidyverse)
library(lubridate)
library(RColorBrewer)
library(ggthemes)
library(ggcorrplot2)
library(psych)
library(caret)
library(ggplot2)
library(tidyr)
library(quantreg)
library(readr)
library(scales)

In [ ]:
### Read file
data <- read.csv("data.csv")
data_ln <- read.csv("data_ln.csv")

In [ ]:
### Trends in variables
data_level <- data
data_log_returns <- data_ln
data_level$Date <- as.Date(data_level$Date, format = "%Y/%m/%d")
level_log_returns <- function(class, pdf_name){
    pdf_path <- paste("/Users/femicind/Documents/Jupyter/03 Climate Finance/output_new/", pdf_name, sep = "")
    pdf(pdf_path, width = 7, height = 4)
    par(las = 1)
    par(pin = c(5.5, 2.5))
    plot(data_level$Date, data_level[,class], type = "l", xaxt = "n", yaxs = "i", xaxs = "i",
         main = class, cex.main = 2, cex.axis = 1.2, xlab = "", ylab = "", lwd = 1, col = "darkblue", lty = 1)  # Dark blue for level
    Date <- seq(from = min(data_level$Date), to = max(data_level$Date), by = "2 years")
    axis.Date(1, at = Date, format = "%Y", cex.axis = 1.2)
    par(las = 1)
    par(new=TRUE)
    plot(data_level$Date, data_log_returns[,class], type = "l", yaxs = "i", xaxs = "i", 
         ylim = c(-20, 20), xlab = "", ylab = "", axes=FALSE, lwd = 1, col = "skyblue", lty = 2)  # Orange for log returns
    axis(side = 4, at = c(-20, -10, 0, 10, 20), labels = c(-20, -10, 0, 10, 20), cex.axis = 1.2)
    legend("topleft", legend=c("Price", "Return"), col=c("darkblue", "skyblue"), lwd=1, lty=c(1, 2))
    dev.off()
}
level_log_returns("Oil", "Oil.pdf")
level_log_returns("Gas", "Gas.pdf")
level_log_returns("Coal", "Coal.pdf")
level_log_returns("Solar", "Solar.pdf")
level_log_returns("Wind", "Wind.pdf")
level_log_returns("Nuclear", "Nuclear.pdf")

In [ ]:
### Descriptive Statistics
DATA <- data_ln
DATA = na.omit(DATA)
DATE = as.Date(as.character(DATA[,1]), format = "%Y/%m/%d")
DATA1 = DATA[,-1]
DATA11 = xts(x = DATA1, order.by = DATE)
return = as.zoo(DATA11)
sum_stat = SummaryStatistics(return, portmanteau="Ljung-Box", correlation="kendall", nlag=10)
sum_stat
write(sum_stat,"sum_stat.csv")

In [ ]:
# BIC&AIC
data_BIC <- data_ln
data_BIC$Date <- as.Date(data_BIC$Date, format="%Y/%m/%d")
data_BIC_clean <- data_BIC[, -1]
VAR_BIC <- VARorder(data_BIC_clean, maxp = 10)
VAR_BIC

In [ ]:
### TVP-VAR
data_H <- data_ln
data_H$Date <- as.Date(data_H$Date, format="%Y/%m/%d")
zoo_data_H <- zoo(data_H[, -1], order.by =data_H$Date)

# for (i in 1:20){
#     dca_TVPVAR = ConnectednessApproach(zoo_data_H, nlag=4, nfore=i, window.size = 252, model="TVP-VAR", connectedness="Time",
#                             VAR_config=list(TVPVAR=list(kappa1=0.99, kappa2=0.96, prior="BayesPrior", gamma = 0.01)))
#     print(dca_TVPVAR$TABLE)
# }

dca_TVPVAR = ConnectednessApproach(zoo_data_H, nlag=4, nfore=10, window.size = 252, model="TVP-VAR", connectedness="Time",
                            VAR_config=list(TVPVAR=list(kappa1=0.99, kappa2=0.96, prior="BayesPrior", gamma = 0.01)))
dca_TVPVAR$TABLE

In [ ]:
TCIC <- rep(0, 2531)
for (i in 1:2531){
    sum = 0
    for (j in 4:6){
        for (k in 4:6){
            if (j != k){
                sum = sum + dca_TVPVAR$CT[j, k, i]
                TCIC[i] = sum / 6 * 100
            }
        }
    }
}
par(las = 1) 
par(pin = c(5.5, 2.5))
plot(TCIC, type = 'l', ylim = c(0,100))
lines(dca_TVPVAR$TCI, col = 2)
write.csv(TCIC, 'TCIC.csv')

In [ ]:
# TCIT_figure
data <- read_csv('TCI_fig.csv')
data$Date <- as.Date(data$Date, format = "%Y/%m/%d")
p <- ggplot(data, aes(x = Date)) +
  geom_line(aes(y = TCIC, color = "TCIC"), size = 0.8) +
  geom_line(aes(y = TCIF, color = "TCIF"), size = 0.8) +
  geom_line(aes(y = TCIT, color = "TCIT"), size = 0.8) +
  scale_color_manual(values = c("TCIC" = "skyblue", "TCIF" = "orange", "TCIT" = "darkblue")) +
  scale_x_date(date_breaks = "1 year", date_labels = "%Y") +

  scale_y_continuous(breaks = seq(0, 80, by = 10)) +
  
  annotate("rect", xmin = as.Date("2020-01-01"), xmax = as.Date("2020-12-31"), 
           ymin = -Inf, ymax = Inf, fill = "grey", alpha = 0.2) +
  annotate("rect", xmin = as.Date("2022-02-24"), xmax = as.Date("2022-12-31"), 
           ymin = -Inf, ymax = Inf, fill = "grey", alpha = 0.2) +
  
  # COVID-19
  annotate("text", x = as.Date("2020-02-10"), y = 68, label = "2020 Covid-19", size = 4, hjust = 0) +
  # 2014.2 Cold wave in the US and Europe
  annotate("text", x = as.Date("2014-01-01"), y = 49, label = "2014.2 Cold wave in \nthe US and Europe", size = 4, hjust = 0) +
  geom_segment(aes(x = as.Date("2014-04-03"), xend = as.Date("2014-02-03"), 
                   y = 46, yend = 41), arrow = arrow(length = unit(0.2, "cm")), color = "black") +
  # 2015.2 Global oil prices plummeted
  annotate("text", x = as.Date("2014-11-02"), y = 61, label = "2015 Global oil \nprices plummeted", size = 4, hjust = 0) +
  geom_segment(aes(x = as.Date("2015-06-04"), xend = as.Date("2015-08-25"), 
                   y = 58, yend = 42), arrow = arrow(length = unit(0.2, "cm")), color = "black") +
  # 2016.6 Global coal capacity and demand fell
  annotate("text", x = as.Date("2016-01-05"), y = 54, label = "2016 Global supply and \ndemand for coal decreased", size = 4, hjust = 0) +
  geom_segment(aes(x = as.Date("2016-08-05"), xend = as.Date("2016-07-05"), 
                   y = 51, yend = 47), arrow = arrow(length = unit(0.2, "cm")), color = "black") +
  # 2018 Renewable energy policy adjustment and technological breakthroughs
  annotate("text", x = as.Date("2017-09-05"), y = 48, label = "2018 Renewable energy \npolicy adjustment and \ntechnological breakthroughs", size = 4, hjust = 0) +
  geom_segment(aes(x = as.Date("2018-03-05"), xend = as.Date("2018-04-09"), 
                   y = 44, yend = 39), arrow = arrow(length = unit(0.2, "cm")), color = "black") +
  # 2019 Global natural gas prices have plummeted
  annotate("text", x = as.Date("2018-10-03"), y = 58, label = "2019 Global natural gas \nprices plummeted", size = 4, hjust = 0) +
  geom_segment(aes(x = as.Date("2019-04-03"), xend = as.Date("2019-06-03"), 
                   y = 55, yend = 41), arrow = arrow(length = unit(0.2, "cm")), color = "black") +
  # 2021 Global natural gas prices have plummeted
  annotate("text", x = as.Date("2021-01-06"), y = 55, label = "2021 Global coal \nprices surged sharply", size = 4, hjust = 0) +
  geom_segment(aes(x = as.Date("2021-06-06"), xend = as.Date("2021-10-06"), 
                   y = 52, yend = 45), arrow = arrow(length = unit(0.2, "cm")), color = "black") +
  # 2022 Russia-Ukraine Conflit
  annotate("text", x = as.Date("2022-03-10"), y = 68, label = "2022 Russia- \nUkraine Conflit", size = 4, hjust = 0) +
  
  # 2024.5 Extreme heatwaves.
  annotate("text", x = as.Date("2024-01-03"), y = 49, label = "2024.5 Extreme \nheatwaves", size = 4, hjust = 0) +
  geom_segment(aes(x = as.Date("2024-04-03"), xend = as.Date("2024-05-03"), 
                   y = 46, yend = 42), arrow = arrow(length = unit(0.2, "cm")), color = "black") +

  labs(
    x = "Date", 
    y = NULL, 
    color = NULL,
    title = NULL
  ) +

  theme_minimal(base_size = 14) +  
  theme(
    axis.text.x = element_text(size = 14, angle = 0, hjust = 1),  
    axis.text.y = element_text(size = 14),                        
    legend.position = c(0.98, 0.98),                             
    legend.justification = c(1, 1),                             
    legend.title = element_blank(),                            
    legend.text = element_text(size = 12),               
    panel.grid = element_blank(),                          
    panel.border = element_rect(color = "black", fill = NA, size = 0.8) 
  )
ggsave(
  filename = "TCI_timeseries_plot_with_events.pdf",  
  plot = p,                                       
  path = "/03 Climate Finance/", 
  width = 16,                                    
  height = 8,                                     
  dpi = 300                                    
)

print(p)

In [ ]:
### NPDC
npdc <- function(){
    data_npdc <- matrix(0, nrow = 6, ncol = 6)
    for (i in 1:6){
        for (j in 1:6){
            sum = 0
            for (k in 1:2531){
                sum <- sum + dca_TVPVAR$NPDC[i ,j, k]
            data_npdc[i, j] = sum / 2531
            }
        } 
    }
    return(data_npdc)
}
data_npdc <- npdc()
data_npdc
write.csv(data_npdc, "NPDC.csv")

In [ ]:
### PCIC
pci <- function(){
    data_pci <- numeric(2531)
    for (i in 1:2531){
        sum <- 0
        for (j in 1:3){
            for (k in 1:3){
                sum <- sum + dca_TVPVAR$PCI[k ,j, i]
            }
        } 
        data_pci[i] = sum - 300
    }
    return(data_pci)
}
data_pci <- pci()
data_pci
write.csv(data_pci, "PCIC.csv")

In [ ]:
### data collection
write.csv(dca_TVPVAR$TABLE, "Table.csv")
write.csv(dca_TVPVAR$NET, "NET_result.csv")
write.csv(dca_TVPVAR$TO, "TO.csv")
write.csv(dca_TVPVAR$FROM, "FROM.csv")
write.csv(dca_TVPVAR$TCI, "TCI.csv")

In [ ]:
df <- read.csv('NET_result.csv')
df$Date <- as.Date(df$Date, format = "%Y/%m/%d")

dt <- pivot_longer(data = df, cols = 2:7, names_to = "Markets", values_to = "net")

dt$Markets <- factor(dt$Markets, levels = c("Solar", "Wind", "Nuclear", "Coal", "Oil", "Gas"))

pdf('NET_photo_separated_v4.pdf', width = 10, height = 7)

plot <- ggplot(data = dt, aes(x = Date, y = net, fill = Markets)) +
  geom_area(alpha = 0.6, position = "stack") +
  scale_fill_viridis_d(option = "C") + 
  labs(x = "Date", y = "Net Spillover", fill = "Markets") +
  facet_wrap(~ Markets, scales = "free_y", nrow = 6, strip.position = "left") +
  scale_x_date(breaks = seq(as.Date("2014-01-01"), max(dt$Date), by = "2 years"), date_labels = "%Y") + 
  theme_minimal() +
  theme(axis.text = element_text(size = 12),
        axis.title = element_text(size = 14),
        axis.text.y = element_text(size = 10),
        strip.text = element_text(size = 12, angle = 0),
        strip.background = element_blank(), 
        panel.grid.major = element_blank(), 
        panel.grid.minor = element_blank(), 
        panel.border = element_blank(),   
        axis.line.x = element_line(size = 0.5),
        axis.line.y = element_line(size = 0.5),
        plot.margin = margin(10, 10, 10, 10), 
        panel.spacing = unit(0.5, "lines")) + 
  geom_hline(yintercept = 0, linetype = "solid", color = "gray", size = 0.3) 

print(plot)
dev.off()

In [ ]:
### TCIT_monthly_median
data <- read.csv("TCIT.csv")
data$Date <- as.Date(data$Date, format = "%Y/%m/%d")
data$YearMonth <- format(data$Date, "%Y-%m")
monthly_median <- aggregate(TCIT ~ YearMonth, data = data, FUN = median)
print(monthly_median)
write.csv(monthly_median,"TCIT_median.csv")

In [ ]:
### Coefficients
data <- read.csv("TCI_median.csv")
y <- data$TCI
x <- data$GCD
c <- data$CRC
d <- data$CPU

quantiles <- 0.05 
taus <- seq(from = quantiles, to = 1 - quantiles, by = quantiles) 

result_matrix <- matrix(NA, nrow = length(taus) + 1, ncol = 9)  # +1 for linear regression results
colnames(result_matrix) <- c("Model", "Intercept", "GCD", "CRC", "CPU", "Intercept_p", "GCD_p", "CRC_p", "CPU_p")
# LR
linear_model <- lm(y ~ x + c + d, data = data)
linear_summary <- summary(linear_model)

result_matrix[1, 1] <- "Linear Regression"
result_matrix[1, 2:5] <- coef(linear_model)  # Coef
result_matrix[1, 6:9] <- coef(summary(linear_model))[, "Pr(>|t|)"]  # p value
# QR
for (i in seq_along(taus)) {
  tau <- taus[i]
  quantile_model <- rq(y ~ x + c + d, data = data, tau = tau, method = "br")
  quantile_summary <- summary(quantile_model, se = "ker")
  
  result_matrix[i + 1, 1] <- paste0("Quantile (tau=", tau, ")")
  result_matrix[i + 1, 2:5] <- coef(quantile_model)  # Coef
  result_matrix[i + 1, 6:9] <- coef(quantile_summary)[, "Pr(>|t|)"]  # p value
}

result_df <- as.data.frame(result_matrix)
write.csv(result_df, "LQR_coef_TCI.csv", row.names = FALSE)

In [ ]:
QR.coef <- summary(rq(y ~ x+c+d, tau = taus, ci = F), se = "ker") 
for (i in 1:num){
  QR.coef[[i]] <- QR.coef[[i]]$coefficients
}
QR.pval <- rep(NA, num)
for (i in 1:num){
  QR.pval[i] <- QR.coef[[i]][2,4]
}
QR.coef1 <- rep(NA, num)
for (i in 1:num){
  QR.coef1[i] <- QR.coef[[i]][2,1]
}
QR.coef <- QR.coef1
rm(QR.coef1)
QR.coef

In [ ]:
# Add QQR Function
lprq <- function(x, y, c, d, m = num, tau = .5) {
  yt <- y
  xt <- x
  ct <- c
  dt <- d
  xx <- seq(min(xt), max(xt), length = m)
  pv <- xx
  dv <- xx
  Fn <- xt
  for (i in 1:length(xt)) {
    Fn[i] <- length(which(xt < xt[i])) / length(xt)
  }
  
  for (i in 1:length(xx)) {
    zt <- xt - xx[i]
    wx <-
      dnorm((Fn - taus[i]) / h) # solve the gaussian kernal as the weight
    r <-
      summary(rq(yt ~ zt+ct+dt,
                 weights = wx,
                 tau = tau,
                 ci = F), se = "ker")  # FALSE method = "br",
    pv[i] <- r$coefficients[2, 4]
    dv[i] <- r$coefficients[2, 1]
  }
  list(xx = xx, pv = pv, dv = dv)
}

#Create a matrix to save the QQR estimates
QQR.coef <- as.data.frame(matrix(0, ncol = num, nrow = num))
QQR.pval <- as.data.frame(matrix(0, ncol = num, nrow = num))

# Run QQR in a loop and save estimates in matrix "QQR.coef"
for (i in 1:num) {
  res <- lprq(x = x,
              y = y,
              c = c,
              d = d,
              m = num,
              tau = taus[i])
  QQR.coef[, i] <- res$dv 
  QQR.pval[, i] <- res$pv
}

#Save the QQR results
# write.csv(QQR.coef, "/Users/femicind/Documents/Jupyter/03 Climate Finance/output_new/data_ln/QQRcoef_PCI_CPU_c.csv", row.names = F)
# write.csv(QQR.pval, "/Users/femicind/Documents/Jupyter/03 Climate Finance/output_new/data_ln/QQRcoef_PCI_CPU_p.csv", row.names = F)

In [ ]:
# Checking QQR
min <- min(c(QR.coef, colMeans(QQR.coef)))
max <- max(c(QR.coef, colMeans(QQR.coef)))
par(las = 1) 
par(pin = c(5.5, 2.5))
pdf('QQRcoef_TCIT_GCD.pdf', width = 7, height = 5)
par(las = 1) 
par(pin = c(5, 2.5))
plot(taus, QR.coef, type = "l", col = "orange", lwd=2, ylab = "Coefficients", xlab = "Quantiles of TCIT", main = "GCD", ylim = c(-0.5, 0.5))
lines(taus, colMeans(QQR.coef), type = "l", lwd=2, lty = 2, col = "blue")
points(taus, QR.coef, pch = 17, col = "orange")
points(taus, colMeans(QQR.coef), pch = 8, col = "blue")
abline(h = 0, col = "gray", lty = 2)
legend("topleft", legend=c("QR", "QQR"), col=c("orange", "blue"), lwd=2, lty = c(1, 2), pch = c(17,8))
dev.off()
plot(taus, QR.coef, type = "l", col = "orange", lwd=2, ylab = "Coefficients", xlab = "Quantiles of TCIT", main = "GCD
", ylim = c(-0.5, 0.5))
lines(taus, colMeans(QQR.coef), type = "l", lwd=2, lty = 2, col = "blue")
points(taus, QR.coef, pch = 17, col = "orange")
points(taus, colMeans(QQR.coef), pch = 8, col = "blue")
abline(h = 0, col = "gray", lty = 2)
legend("topleft", legend=c("QR", "QQR"), col=c("orange", "blue"), lwd=2, lty = c(1, 2), pch = c(17,8))